# The CMS ECAL Crystals: An Electromagnetic Analysis
## ***Walker Hendricks***
---
## Introduction
The CMS (Compact Muon Solenoid) is a complicated experiment at the Large Hadron Collider (LHC) at CERN in Geneva, Switzerland. It is one of the two major experiments along the LHC, being diametrically opposite to the other, ATLAS. The goal of this experiment is to measure the energy-momentum four-vectors of particles emerging from $pp$ collisions, which requires very technical equipment. One component of the CMS is the **electromagnetic calorimeter**, or **ECAL**. This component attempts to measure the four-vectors of the smaller QED particles: the electron, the positron, and the photon. 

## Our Model

The structure of the crystals is outlined in the [CMS TDR (technical design report)](https://jinst.sissa.it/LHC/CMS/ch04.pdf). They are square pyramidal frusta with a front face of 22x22mm$^2$ and a back face of 26x26mm$^2$, made of the heavy metal oxide lead tungstate ($\textrm{PbWO}_4$). Based on the TDR, they appear to be bounded by a thin layer of air. In the TDR, it describes the surroundings of the crystal as follows:

> The centres of the front faces of the crystals are at a radius 1.29 m. The crystals are contained
in a thin-walled alveolar structure (submodule). The alveolar wall is 0.1 mm thick and is made of an
aluminium layer, facing the crystal, and two layers of glass fibre-epoxy resin. To avoid oxidation,
a special coating is applied to the aluminium surface. The nominal crystal to crystal distance is
0.35 mm inside a submodule, and 0.5 mm between submodules

From my understanding, this appears to be a 0.35mm crystal-to-crystal separation with two 0.1mm alveolar walls in between. This then leaves a gap of (0.35/2) - 0.1 mm = 75$\mu$m around each crystal, presumably of air. Given that the index of refraction of the crystals is $n_{\textrm{PbWO}_4} \sim 2.29$ and the index of refraction for air is $n_{air} \sim 1$, we will have total internal reflection here. Thus, the CMS ECAL crystals act as optical fibers, funneling the light down the crystal to the detector at the end (a SiPM).

While for the purposes of the CMS a fiber-like description of the material is suitable, a geometric ray optics approach breaks down for a fiber when the wavelength of the incident light is comparable to the transverse size of the fiber itself. This of course is not a problem for the CMS ECAL, but it *does* pose an interesting area of examination for the purposes of me getting a good grade on this E&M assignment! ;)

Thus, this notebook is broken into two parts: the first part will deal with the ECAL crystal from a geometric/ray optics approach, looking at things like ray tracing and total internal reflection, while the second part will deal with finding the normal modes of the dielectric crystal. Let's get started!

## Imports

We will use the following imports for this project:

`pyvista`: This is what allows us to have the 3D animation/plotting

`numpy`: Standard numerical calculations

`ipywidgets` and `IPython.display`: These packages are used with Jupyter for making widgets

`plotly`: This is used to make the "impact pad" for determining where on the crystal face the light beam hits

If you have not installed these packages previously, you can install them with the line below (make sure you're in the environment you like!):

In [ ]:
# Installing required packages (console)
!pip install numpy pyvista trame trame-vuetify trame-vtk plotly anywidget

Note that if you used the above line, you will need to restart Jupyter before continuing (not just the kernel, but Jupyter itself).

Now import the packages below:

In [ ]:
# Import statements (Python)
import pyvista as pv
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

## Part 1: The Geometric Approach
---
Recall that for total internal reflection, light incident at angles below the critial angle
$$
\theta_c = \cos^{-1}  \left(\frac{n_0}{n_1} \right)
$$
will be totally reflected, while light incident at angles greater than this will only have partial reflection. Of course, $\theta$ here is defined as the angle off of the central axis of the fiber/waveguide. The smaller this angle $\theta$, the more parallel the ray is to the sides of the fiber/waveguide. 



Below, I will begin creating the geometry for our visualization. The crystal is formed by specifying the vertices on a PyVista hexahedron shape. This way I can define the difference in size between the front and back faces of the crystal.

In [ ]:
# Crystal parameters -- tapers from 22x22 mm at the front (base) to 26x26 mm at the back (top)
L = 230.0      # Length along Z
w_base = 22.0  # Width of the front face (z=0)
h_base = 22.0  # Height of the front face (z=0)
w_top = 26.0   # Width of the back face (z=L)
h_top = 26.0   # Height of the back face (z=L)

# Defining hexahedron vertices
vertices = np.array([
    # Base (z=0)
    [-w_base/2, -h_base/2, 0],
    [+w_base/2, -h_base/2, 0],
    [+w_base/2, +h_base/2, 0],
    [-w_base/2, +h_base/2, 0],
    
    # Top (z=L)
    [-w_top/2, -h_top/2, L],  
    [+w_top/2, -h_top/2, L],  
    [+w_top/2, +h_top/2, L],  
    [-w_top/2, +h_top/2, L]   
])

# Defining connectivity
cells_flat = np.array([
    8,                      # One cell has 8 vertices
    0, 1, 2, 3, 4, 5, 6, 7  # Vertices indices
])

# Defining cell as VTK_HEXAHEDRON
cell_types = np.array([pv.CellType.HEXAHEDRON])

# Creating mesh
crystal = pv.UnstructuredGrid(cells_flat, cell_types, vertices)


# Plotting crystal
pl = pv.Plotter(window_size=[800, 600], notebook=True)
pl.add_mesh(
    crystal, 
    color='green', 
    opacity=0.75, 
    show_edges=True, 
    line_width=3
)
pl.add_axes(xlabel='X (mm)', ylabel='Y (mm)', zlabel='Z (mm)')
pl.show(jupyter_backend='trame')

Notice how you can observe the fact that the crystal is a frustum by turning on the bounding box. On one end of the crystal, the crystal completely fills the bounding box; on the other, it has shrunk noticeably (but only really with the bounding box turned on). 

Now that the geometry is made, let's work on the GUI next. I'm aiming for three editable parameters: the impact position on the crystal face and the spherical angles $\phi$ and $\theta$ (where the $z$-axis is down the crystal). The angular variations are simple enough--I'll just add some sliders for those. For the impact position, I envision a 2D grid where you can click to select where the ray will enter the crystal at. I also want a button we can click to begin the raytracing calculations (so your computer doesn't die when you slide a bar to quickly). So let's make these GUI elements here.

In [ ]:
# Initial State
impact_state = {'x': 0, 'y': 0}

# GUI elements

# Impact pad (plotly)
N_GRID = 500

impact_pad = go.FigureWidget()

# Adding the heatmap
z_data = np.zeros((N_GRID, N_GRID)) 
x_coords = np.linspace(-w_base/2, w_base/2, N_GRID)
y_coords = np.linspace(-h_base/2, h_base/2, N_GRID)

impact_pad.add_trace(go.Heatmap(
    z=z_data,
    x=x_coords,
    y=y_coords,
    colorscale=[[0, 'rgba(211,211,211,0.5)'], [1, 'rgba(211,211,211,0.5)']],
    showscale=False,
    name='Crystal Face',
    hoverinfo='none' 
))

# Red dot for impact position
impact_pad.add_trace(go.Scatter(
    x=[0],
    y=[0],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Impact Point',
    hovertemplate='<b>Impact Point</b><br>x: %{x:.2f} mm<br>y: %{y:.2f} mm<extra></extra>'
))

# Defining the main graph/2D space
impact_pad.update_layout(
    title='Click to Set Impact Position',
    width=300, height=300,
    xaxis=dict(
        range=[-w_base/2 - 2, w_base/2 + 2], 
        scaleanchor="y", 
        scaleratio=1,
        fixedrange=True
    ),
    yaxis=dict(
        range=[-h_base/2 - 2, h_base/2 + 2],
        fixedrange=True
    ),
    showlegend=False,
    margin=dict(l=10, r=10, b=10, t=40)
)

# On-click callback function
def update_impact_point(trace, points, selector):    
    # Check if we actually have click coordinates
    if points.xs:
        x = points.xs[0]
        y = points.ys[0]
        
        # Update global state
        impact_state['x'] = x
        impact_state['y'] = y
        
        # Use batch_update for redrawing
        with impact_pad.batch_update():
            impact_pad.data[1].x = [x]
            impact_pad.data[1].y = [y]

# Link callback to heatmap
impact_pad.data[0].on_click(update_impact_point)


# IPyWidgets (sliders and buttons)
theta_slider = widgets.FloatSlider(
    value=0, min=-90, max=90, step=1, 
    description='Theta (θ):',
    style={'description_width': 'initial'}
)
phi_slider = widgets.FloatSlider(
    value=0, min=0, max=360, step=1, 
    description='Phi (φ):',
    style={'description_width': 'initial'}
)
raytrace_button = widgets.Button(
    description='Generate Rays',
    button_style='success'
)

# Using a vertical box (VBox) to assemble the widgets
controls = widgets.VBox([
    impact_pad,
    theta_slider,
    phi_slider,
    raytrace_button
])

# Displaying GUI controls
display(controls)

Now that the widgets appear to be in proper working order, we can proceed to work on the actual raytracing code. One interesting feature I would like to implement is a change in the ray's alpha value when the angle exceeds the critical angle and light escapes the crystal. To find this, we can use the **Fresnel Equations**. What we will do is use these relationships for both waves parallel and perpendicular to the plane of incidence and then average them out for an arbitrary unpolarized wave. To further explain what I mean, consider the Fresnel equations for a wave polarized *perpendicular* to the plane of incidence,
$$
\frac{E_T}{E_0}=\frac{2n_1\cos\theta_i}{n_1\cos\theta_i + \sqrt{n_2^2-n_1^2\sin^2\theta_i}} \hspace{1cm} \frac{E_R}{E_0}=\frac{n_1\cos\theta_i - \sqrt{n_2^2-n_1^2\sin^2\theta_i}}{n_1\cos\theta_i + \sqrt{n_2^2-n_1^2\sin^2\theta_i}}
$$
and *parallel* to the plane of incidence,
$$
\frac{E_T}{E_0}=\frac{2n_1n_2\cos\theta_i}{n_2^2\cos\theta_i + n_1\sqrt{n_2^2-n_1^2\sin^2\theta_i}} \hspace{1cm} \frac{E_R}{E_0}=\frac{n_2^2\cos\theta_i - n_1\sqrt{n_2^2-n_1^2\sin^2\theta_i}}{n_2^2\cos\theta_i + n_1\sqrt{n_2^2-n_1^2\sin^2\theta_i}}
$$
where the subscripts $T$ and $R$ indicate the *transmitted* and *reflected* portions of the wave, respectively; $\theta_i$ indicates the incident angle. Note also that we are assuming the materials involved are not magnetic ($\mu_1=\mu_2=\mu_0$). Now, for overall reflection and transmission coefficients, we can average the intensity of these two polarizations:
$$
T = \frac{(E_{T\perp})^2+(E_{T||})^2}{2(E_0)^2} \hspace{1cm} R = \frac{(E_{R\perp})^2+(E_{R||})^2}{2(E_0)^2}
$$
Of course, having found one of the coefficients, we can use conservation of energy to determine the other:
$$
T + R = 1
$$
We will code up the relationship for $R$ and, obtaining that, we will calculate $T=1-R$. We can then use this to change the alpha values appropriately, showing light intensity diminishing as it refracts beyond the critical angle and escapes the crystal.

In [ ]:
import numpy as np

# Constants
N_CRYSTAL = 2.2 
N_AIR = 1.0      

# Calculating slope for wall normal determination
wall_slope = (w_top - w_base) / (2 * L)

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: return v
    return v / norm

def get_wall_normal(side):
    """
    Returns the inward-pointing normal vector for a given side.
    side 0: Right (+x), 1: Left (-x), 2: Top (+y), 3: Bottom (-y)
    """
    if side == 0: return normalize(np.array([-1.0, 0.0, wall_slope])) 
    if side == 1: return normalize(np.array([ 1.0, 0.0, wall_slope])) 
    if side == 2: return normalize(np.array([0.0, -1.0, wall_slope])) 
    if side == 3: return normalize(np.array([0.0,  1.0, wall_slope])) 
    return np.array([0,0,-1])

def intersect_plane(ray_origin, ray_dir, point_on_plane, plane_normal):
    """
    Calculates distance 't' to a plane defined by a point and normal.
    Returns infinity if parallel or behind the ray.
    """
    denom = np.dot(ray_dir, plane_normal)
    if abs(denom) < 1e-6: return np.inf 
    
    vector_to_plane = point_on_plane - ray_origin
    t = np.dot(vector_to_plane, plane_normal) / denom
    
    # Returns t if valid intersection (epsilon prevents self-intersection)
    return t if t > 1e-4 else np.inf 

def get_fresnel_transmission(n1, n2, normal, incident):
    """
    Calculates the Fraction Lost (Transmittance) using derived formulas.
    Returns (Transmittance T, Reflection Vector)
    """    
    # Calculating angle of incidence
    cos_i = np.dot(-incident, normal)
    if cos_i < 0: cos_i = 0 
    theta_i = np.arccos(cos_i)
    
    # Checking for total internal reflection
    if n2 < n1: 
        theta_c = np.arcsin(n2 / n1)
        if theta_i >= theta_c:
            reflect_dir = incident + 2 * cos_i * normal
            return 0.0, reflect_dir 

    # Calculating transmission angle via Snell's Law
    sin_t = (n1 / n2) * np.sin(theta_i)
    theta_t = np.arcsin(sin_t)
    
    # Fresnel equations for S and P polarization
    num_s = n1 * np.cos(theta_i) - n2 * np.cos(theta_t)
    den_s = n1 * np.cos(theta_i) + n2 * np.cos(theta_t)
    R_s = (num_s / den_s)**2
    
    num_p = n2 * np.cos(theta_i) - n1 * np.cos(theta_t)
    den_p = n2 * np.cos(theta_i) + n1 * np.cos(theta_t)
    R_p = (num_p / den_p)**2
    
    # Averaging for unpolarized light
    R_eff = (R_s + R_p) / 2.0
    T_eff = 1.0 - R_eff
    
    reflect_dir = incident + 2 * cos_i * normal
    
    return T_eff, reflect_dir

def calculate_ray_path(start_pos, theta, phi):
    """
    Traces a ray through the crystal.
    Returns a list of segments: [(p_start, p_end, intensity), ...]
    """
    segments = []
    
    # Converting spherical angles to direction vector
    theta_rad = np.deg2rad(theta)
    phi_rad = np.deg2rad(phi)
    
    dx = np.sin(theta_rad) * np.cos(phi_rad)
    dy = np.sin(theta_rad) * np.sin(phi_rad)
    dz = np.cos(theta_rad) 
    
    direction = normalize(np.array([dx, dy, dz]))
    
    # Backtracking to create a visual source outside the crystal
    visual_start_z = -40.0
    if abs(dz) > 1e-6:
        t_back = visual_start_z / dz
        source_pos = start_pos + t_back * direction
        segments.append((source_pos, start_pos, 1.0)) 
    
    # Setting up trace variables
    current_pos = start_pos.astype(float)
    current_dir = direction
    current_intensity = 1.0
    
    # Max 50 bounces to prevent infinite loops
    for _ in range(50): 
        candidates = []
        
        # calculating intersections for side walls
        # Side 0: Right
        norm_r = get_wall_normal(0)
        pt_r = np.array([w_base/2, 0, 0]) 
        candidates.append((intersect_plane(current_pos, current_dir, pt_r, -norm_r), 'side', norm_r))
        
        # Side 1: Left
        norm_l = get_wall_normal(1)
        pt_l = np.array([-w_base/2, 0, 0])
        candidates.append((intersect_plane(current_pos, current_dir, pt_l, -norm_l), 'side', norm_l))
        
        # Side 2: Top
        norm_t = get_wall_normal(2)
        pt_t = np.array([0, w_base/2, 0])
        candidates.append((intersect_plane(current_pos, current_dir, pt_t, -norm_t), 'side', norm_t))
        
        # Side 3: Bottom
        norm_b = get_wall_normal(3)
        pt_b = np.array([0, -w_base/2, 0])
        candidates.append((intersect_plane(current_pos, current_dir, pt_b, -norm_b), 'side', norm_b))
        
        # Back Face
        norm_back = np.array([0, 0, -1]) 
        pt_back = np.array([0, 0, L])
        candidates.append((intersect_plane(current_pos, current_dir, pt_back, norm_back), 'back', norm_back))
        
        # Filtering and sorting for closest valid hit
        valid_hits = [c for c in candidates if c[0] != np.inf]
        if not valid_hits: break 
        
        valid_hits.sort(key=lambda x: x[0])
        t_hit, type_hit, normal_hit = valid_hits[0]
        
        next_pos = current_pos + t_hit * current_dir
        segments.append((current_pos, next_pos, current_intensity))
        
        # Handling interactions
        if type_hit == 'back':
            break
            
        elif type_hit == 'side':
            T, reflect_dir = get_fresnel_transmission(N_CRYSTAL, N_AIR, normal_hit, current_dir)
            
            # Visualizing light leakage if transmission is significant
            if T > 0.01: 
                leak_end = next_pos + 10.0 * normal_hit 
                segments.append((next_pos, leak_end, current_intensity * T)) 
            
            current_intensity *= (1.0 - T)
            current_dir = reflect_dir
            current_pos = next_pos
            
            # Stopping trace if intensity is too low
            if current_intensity < 0.05: 
                break
                
    return segments

Now we'll put it all together down here:

In [ ]:
# Simulation logic callback
def run_simulation(b):
    # Clearing previous ray segments
    for name in list(pl.actors.keys()):
        if 'ray_segment' in name:
            pl.remove_actor(name)
            
    # Reusing globals: impact_state, theta_slider, phi_slider
    x = impact_state['x']
    y = impact_state['y']
    theta = theta_slider.value
    phi = phi_slider.value
        
    start_point = np.array([x, y, 0.0])
    segments = calculate_ray_path(start_point, theta, phi)
    
    # Drawing new rays
    for i, (p_start, p_end, intensity) in enumerate(segments):
        line = pv.Line(p_start, p_end)
        pl.add_mesh(
            line, 
            color='red', 
            line_width=2, 
            opacity=intensity, 
            name=f'ray_segment_{i}'
        )

# Linking button (reusing raytrace_button from previous cell)
raytrace_button.on_click(run_simulation)

# Fetching the plotter viewer again to arrange it side-by-side
plot_output = pl.show(jupyter_backend='trame', return_viewer=True)

# Assembling the final application layout using existing 'controls'
app = widgets.HBox([plot_output, controls])

display(app)

## Part 2: Finding the Normal Modes
---
(Not) Coming soon

See `meep` package